In [5]:
from tensorflow.keras.layers import *
from tensorflow.keras import Model

In [25]:
def identity_block(x, filters):
   
    f1, f2, f3 = filters

    x1 = x
   
    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)

    x = Add()([x, x1])

    return x

In [26]:
def convolutional_block(x, filters, s=2):

    f1, f2, f3 = filters

    x1 = x

    x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(s, s))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1))(x)
    x = BatchNormalization()(x)

    x1 = Conv2D(filters=f3, kernel_size=(1, 1), strides=(s, s))(x1)
    x1 = BatchNormalization()(x1)

    x = Add()([x, x1])
    X = Activation('relu')(x)

    return X

In [27]:
def ResNet50(input_shape=(224, 224, 3), num_classes=1000):

    x_input = Input(input_shape)

    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same')(x_input)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = convolutional_block(x, [64, 64, 256], s=1)
    x = identity_block(x, [64, 64, 256])
    x = identity_block(x, [64, 64, 256])

    x = convolutional_block(x, [128, 128, 512], s=2)
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])
    x = identity_block(x, [128, 128, 512])

    x = convolutional_block(x, [256, 256, 1024], s=2)
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])
    x = identity_block(x, [256, 256, 1024])

    x = convolutional_block(x, [512, 512, 2048], s=2)
    x = identity_block(x, [512, 512, 2048])
    x = identity_block(x, [512, 512, 2048])

    x = GlobalAveragePooling2D()(x)
    x_output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=x_input, outputs=x_output, name='ResNet50')

    return model

In [28]:
ResNet50().summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_220 (Conv2D)            (None, 112, 112, 64  9472        ['input_7[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_220 (Batch  (None, 112, 112, 64  256        ['conv2d_220[0][0]']             
 Normalization)                 )                                                          

In [24]:
import tensorflow as tf
tf.keras.applications.resnet50.ResNet50(
    include_top=True,
    weights='imagenet'
).summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_6[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           